In [1]:
### Make CSV of CONUS2 flow at matched gages ###
### DTT, 05/23

# This script is split into two main parts: 1) reading in CONUS2 gridded total (aggregated) daily flow for the full domain and creating a dataframe of CONUS2 flow for cells that have been matched with USGS gages in the `NWM_Gage_Adjustments_final.csv`. 2) matching the gages that both have flow between the PF csv and USGS csv retrieved from hydrodata. 
# Note that flow is converted in this script from daily accumulated flow in [m^3/h] to daily mean flow in cms or [m^3/s].

### Inputs:
# - `NWM_Gage_Adjustments_final.csv` - this can be found on the CONUS2 Dropbox or in /glade/p/univ/ucsm0002/CONUS2/domain_files
# - Daily total streamflow PFCLM outputs as PFBs - processed using `compute_daily_PF_averages.py`
# - USGS daily flow csv - from the hydrodata repository on Verde

### Outputs:
# - CSV of PFCLM daily mean flow (in Cubic Meters per Second) with gage ID, lat/long, and CONUS2 cell location
# - two flow-matched CSVs for PF and USGS flow
# - note that the CSV outputs with 'day 001' which starts at the water year (001 == October 1)***

# Notes:
# - need to fix the no_days, currently this will only be accurate if this is started at the begninning of a water year and need to add in some dictionary or if statement to specify num days in a month or something like that.
# - ***need to change day headings so that they are more descriptive than 'day 001' and have an actual date

import sys
from parflow.tools.io import read_pfb,write_pfb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime


### Directory where PF flow PFBs are saved in ###
## Cheyenne
#directory_out = '/glade/p/univ/ucsm0002/CONUS2/CONUS2.spinup.WY2003/averages'
#organized_dir = '/glade/work/tijerina/PFCONUS2-analysis/scripts/Validation/Streamflow/Organized_Daily_Flow'
## Hydrodata
directory_out = '/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages'
organized_dir = './Organized_LH'

obs_data_file = 'Hydrodata_latent-heat-flux_obs_hourly_total_WY2003.csv' #csv of USGS flow from hydrodata
metadata_file = 'Hydrodata_latent-heat-flux_metadata_hourly_total_WY2003.csv' #csv of USGS flow from hydrodata

ny = 3256
nx = 4442

# need to change water year and number of days
water_year = 2003
no_days = 364 

# date variables for datetime for wateryear
start_date = datetime.date(water_year, 10, 1)
end_date = datetime.date(water_year, 9, 30)
# daily time step for reading daily files (this could change depending on how you are analyzing)
# delta time
delta = datetime.timedelta(days=1)

In [4]:
# Read observation data and organize
obs_data = pd.read_csv(f'{organized_dir}/{obs_data_file}', index_col=['site_id'])
obs_data = obs_data.drop(columns=['Unnamed: 0'])
# # remove sites with less than 365 days of observations
# obs_data = obs_data.loc[(obs_data['num_obs']==365)]
# Read metadata and organize
metadata = pd.read_csv(f'{organized_dir}/{metadata_file}', index_col=['site_id'])
metadata = metadata.drop(columns=['Unnamed: 0'])
# also remove the sites with less than 365 obs from the metadata
metadata = metadata[metadata.index.isin(obs_data.index)]

# add number of observations column from the obs_data df
metadata['num_obs'] = obs_data['num_obs']
# remove num_obs from data so we can sum and calc stats
obs_data = obs_data.drop(columns=['num_obs'])
obs_data

,2002-10-01 00:00:00,2002-10-01 01:00:00,2002-10-01 02:00:00,2002-10-01 03:00:00,2002-10-01 04:00:00,2002-10-01 05:00:00,2002-10-01 06:00:00,2002-10-01 07:00:00,2002-10-01 08:00:00,2002-10-01 09:00:00,...,2003-09-29 15:00:00,2003-09-29 16:00:00,2003-09-29 17:00:00,2003-09-29 18:00:00,2003-09-29 19:00:00,2003-09-29 20:00:00,2003-09-29 21:00:00,2003-09-29 22:00:00,2003-09-29 23:00:00,2003-09-30 00:00:00
site_id,,,,,,,,,,,,,,,,,,,,,
US-ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
US-Blo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,81.698000,86.234000,249.731000,228.428000,260.023000,271.887000,317.925000,316.995000,169.443000,134.388000
US-Bo1,3.000000,-1.000000,-3.000000,-2.000000,0.000000,-7.000000,1.000000,-6.000000,-6.000000,0.000000,...,159.000000,165.000000,202.000000,207.000000,85.000000,159.000000,129.000000,84.000000,33.000000,14.000000
US-CMW,71.359402,12.017639,-13.517526,-15.882821,10.088744,25.006290,2.864254,-32.073075,1.527340,-0.757087,...,401.584104,611.891544,597.124792,687.145139,619.868660,765.009194,521.543262,428.275266,302.769649,59.134963
US-Cop,23.545000,12.115000,19.318000,20.190000,4.659000,15.651000,14.044000,13.244000,10.372000,5.184000,...,7.583000,14.748000,18.875000,17.060000,17.502000,14.541000,13.846000,9.938000,4.663000,2.174000
US-Dk1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,389.120000,420.340000,404.180000,413.740000,324.440000,284.430000,150.760000,4.406000,-10.362000,-0.269000
US-Dk2,-5.257000,1.389000,8.370000,-0.776000,2.366000,3.876000,12.231000,-1.407000,-0.790000,-1.118000,...,318.280000,240.060000,320.880000,254.160000,229.425000,330.625000,161.128000,35.272000,-5.248000,20.130000
US-Dk3,-1.214000,2.062000,1.726000,6.202000,152.107000,-8.449000,0.397000,-169.540000,-17.505000,-11.044000,...,352.930000,351.110000,330.150000,365.930000,269.620000,383.070000,58.060000,37.313000,-14.458000,-4.576000
US-GBT,57.383000,16.060000,20.064000,16.409000,9.397000,7.879000,6.932000,1.525000,2.223000,-0.262000,...,43.262000,65.246000,81.290000,70.601000,55.896000,98.452000,124.522000,42.920000,21.389000,6.531000


In [3]:
metadata

,site_type,agency,site_name,site_id_agency,site_query_url,date_metadata_last_updated,latitude,longitude,state,huc,...,site_funding,acknowledgement,acknowledgement_comment,doi_citation,alternate_url,conus1_x,conus1_y,conus2_x,conus2_y,num_obs
site_id,,,,,,,,,,,,,,,,,,,,,
US-ARM,flux tower,AmeriFlux,ARM Southern Great Plains site- Lamont,US-ARM,https://ameriflux.lbl.gov/sites/siteinfo/US-ARM,2023-05-02,36.6058,-97.4888,OK,NaN,...,DOE/ARM,This research was supported by the Office of B...,NaN,"Sebastien Biraud, Marc Fischer, Stephen Chan, ...",http://www.arm.gov/sites/sgp,1752.0,342.0,2165,1301,6523
US-Blo,flux tower,AmeriFlux,Blodgett Forest,US-Blo,https://ameriflux.lbl.gov/sites/siteinfo/US-Blo,2023-05-02,38.8953,-120.6328,CA,NaN,...,"DOE/NIGEC, EPA, University of California",NaN,NaN,"Allen Goldstein (2019), AmeriFlux BASE US-Blo ...",http://www.cnr.berkeley.edu/~ahg/tgbl/group.html,NaN,NaN,248,1840,8737
US-Bo1,flux tower,AmeriFlux,Bondville,US-Bo1,https://ameriflux.lbl.gov/sites/siteinfo/US-Bo1,2023-05-02,40.0062,-88.2904,IL,NaN,...,NOAA/GEWEX,NaN,NaN,"Tilden Meyers (2016), AmeriFlux BASE US-Bo1 Bo...",http://www.life.illinois.edu/bernacchi/,2539.0,744.0,2926,1708,8737
US-CMW,flux tower,AmeriFlux,Charleston Mesquite Woodland,US-CMW,https://ameriflux.lbl.gov/sites/siteinfo/US-CMW,2023-05-02,31.6637,-110.1777,AZ,NaN,...,"USDA, NSF",NaN,NaN,"Russell Scott (2022), AmeriFlux BASE US-CMW Ch...",NaN,NaN,NaN,975,862,8737
US-Cop,flux tower,AmeriFlux,Corral Pocket,US-Cop,https://ameriflux.lbl.gov/sites/siteinfo/US-Cop,2023-05-02,38.0900,-109.3900,UT,NaN,...,University of Utah and USGS,NaN,NaN,"David Bowling (2019), AmeriFlux BASE US-Cop Co...",http://esp.cr.usgs.gov/info/sw/clim-met/corral...,720.0,591.0,1156,1544,8737
US-Dk1,flux tower,AmeriFlux,Duke Forest-open field,US-Dk1,https://ameriflux.lbl.gov/sites/siteinfo/US-Dk1,2023-05-02,35.9712,-79.0934,NC,NaN,...,DOE/TCP,NaN,NaN,"Chris Oishi, Kim Novick, Paul Stoy (2018), Ame...",http://www.nicholas.duke.edu/other/AMERIFLUX/a...,NaN,NaN,3772,1408,8737
US-Dk2,flux tower,AmeriFlux,Duke Forest-hardwoods,US-Dk2,https://ameriflux.lbl.gov/sites/siteinfo/US-Dk2,2023-05-02,35.9736,-79.1004,NC,NaN,...,DOE/NIGEC,NaN,NaN,"Chris Oishi, Kim Novick, Paul Stoy (2018), Ame...",http://www.nicholas.duke.edu/other/AMERIFLUX/a...,NaN,NaN,3771,1408,8737
US-Dk3,flux tower,AmeriFlux,Duke Forest - loblolly pine,US-Dk3,https://ameriflux.lbl.gov/sites/siteinfo/US-Dk3,2023-05-02,35.9782,-79.0942,NC,NaN,...,DOE/TCP,NaN,NaN,"Chris Oishi, Kim Novick, Paul Stoy (2018), Ame...",http://www.nicholas.duke.edu/other/AMERIFLUX/a...,NaN,NaN,3771,1408,8737
US-GBT,flux tower,AmeriFlux,GLEES Brooklyn Tower,US-GBT,https://ameriflux.lbl.gov/sites/siteinfo/US-GBT,2023-05-02,41.3658,-106.2397,WY,NaN,...,USDA/Forest Service,NaN,NaN,"Bill Massman (2016), AmeriFlux BASE US-GBT GLE...",http://www.fs.fed.us/rmrs/experimental-forests...,1034.0,915.0,1463,1859,8737


In [5]:
obs_data

,2002-10-01 00:00:00,2002-10-01 01:00:00,2002-10-01 02:00:00,2002-10-01 03:00:00,2002-10-01 04:00:00,2002-10-01 05:00:00,2002-10-01 06:00:00,2002-10-01 07:00:00,2002-10-01 08:00:00,2002-10-01 09:00:00,...,2003-09-29 15:00:00,2003-09-29 16:00:00,2003-09-29 17:00:00,2003-09-29 18:00:00,2003-09-29 19:00:00,2003-09-29 20:00:00,2003-09-29 21:00:00,2003-09-29 22:00:00,2003-09-29 23:00:00,2003-09-30 00:00:00
site_id,,,,,,,,,,,,,,,,,,,,,
US-ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
US-Blo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,81.698000,86.234000,249.731000,228.428000,260.023000,271.887000,317.925000,316.995000,169.443000,134.388000
US-Bo1,3.000000,-1.000000,-3.000000,-2.000000,0.000000,-7.000000,1.000000,-6.000000,-6.000000,0.000000,...,159.000000,165.000000,202.000000,207.000000,85.000000,159.000000,129.000000,84.000000,33.000000,14.000000
US-CMW,71.359402,12.017639,-13.517526,-15.882821,10.088744,25.006290,2.864254,-32.073075,1.527340,-0.757087,...,401.584104,611.891544,597.124792,687.145139,619.868660,765.009194,521.543262,428.275266,302.769649,59.134963
US-Cop,23.545000,12.115000,19.318000,20.190000,4.659000,15.651000,14.044000,13.244000,10.372000,5.184000,...,7.583000,14.748000,18.875000,17.060000,17.502000,14.541000,13.846000,9.938000,4.663000,2.174000
US-Dk1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,389.120000,420.340000,404.180000,413.740000,324.440000,284.430000,150.760000,4.406000,-10.362000,-0.269000
US-Dk2,-5.257000,1.389000,8.370000,-0.776000,2.366000,3.876000,12.231000,-1.407000,-0.790000,-1.118000,...,318.280000,240.060000,320.880000,254.160000,229.425000,330.625000,161.128000,35.272000,-5.248000,20.130000
US-Dk3,-1.214000,2.062000,1.726000,6.202000,152.107000,-8.449000,0.397000,-169.540000,-17.505000,-11.044000,...,352.930000,351.110000,330.150000,365.930000,269.620000,383.070000,58.060000,37.313000,-14.458000,-4.576000
US-GBT,57.383000,16.060000,20.064000,16.409000,9.397000,7.879000,6.932000,1.525000,2.223000,-0.262000,...,43.262000,65.246000,81.290000,70.601000,55.896000,98.452000,124.522000,42.920000,21.389000,6.531000


In [ ]:
# make CONUS2 x and y into arrays for the for loop
conusy = np.asarray(metadata['conus2_y'],dtype = 'int')
conusx = np.asarray(metadata['conus2_x'],dtype = 'int')

# Set up arrays 
pf_flow_array = np.zeros((no_days, ny, nx))
pf_flow_matched = np.zeros(obs_data.shape)

### READ STREAMFLOW PFBs ###
#Read in CONUS2 daily streamflow PFBs and save as df in flow_df, convert to total accumulated in m^3/h to mean daily in cms
for i in range(no_days):
    step = str(int(i+1)).rjust(3, '0')
    print(f'{directory_out}/eflx_lh_tot.2003.daily.{step}.pfb')
    pf_flow_pfb = np.squeeze(read_pfb(f'{directory_out}/eflx_lh_tot.2003.daily.{step}.pfb'))
    pf_flow_array[i,...] = pf_flow_pfb
    print(f'reading PF-CONUS2 output for day {step}')
    for j in range(len(obs_data.index)):
        if conusy[j] < 0:
            pf_flow_matched[j] = 'nan'
            #print('Gage is outside of CONUS2 range')
        else:
            pf_flow_matched[j,i] = pf_flow_array[i, conusy[j], conusx[j]]
            #print('Finding value in CONUS2 grid')
clm_output += read_pfb(f'{path_outputs}{runname}.out.clm_output.{timestamp_reading}.C.pfb')

In [6]:
# make CONUS2 x and y into arrays for the for loop
conusy = np.asarray(metadata['conus2_y'],dtype = 'int')
conusx = np.asarray(metadata['conus2_x'],dtype = 'int')

# Set up arrays 
pf_flow_array = np.zeros((no_days, ny, nx))
pf_flow_matched = np.zeros(obs_data.shape)

### READ STREAMFLOW PFBs ###
#Read in CONUS2 daily streamflow PFBs and save as df in flow_df, convert to total accumulated in m^3/h to mean daily in cms
for i in range(no_days):
    step = str(int(i+1)).rjust(3, '0')
    print(f'{directory_out}/eflx_lh_tot.2003.daily.{step}.pfb')
    pf_flow_pfb = np.squeeze(read_pfb(f'{directory_out}/eflx_lh_tot.2003.daily.{step}.pfb'))
    pf_flow_array[i,...] = pf_flow_pfb
    print(f'reading PF-CONUS2 output for day {step}')
    for j in range(len(obs_data.index)):
        if conusy[j] < 0:
            pf_flow_matched[j] = 'nan'
            #print('Gage is outside of CONUS2 range')
        else:
            pf_flow_matched[j,i] = pf_flow_array[i, conusy[j], conusx[j]]
            #print('Finding value in CONUS2 grid')

/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages/swe_out.2003.daily.001.pfb
reading flow for day 001 and converting from m^3/h to m^3/s
/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages/swe_out.2003.daily.002.pfb
reading flow for day 002 and converting from m^3/h to m^3/s
/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages/swe_out.2003.daily.003.pfb
reading flow for day 003 and converting from m^3/h to m^3/s
/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages/swe_out.2003.daily.004.pfb
reading flow for day 004 and converting from m^3/h to m^3/s
/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages/swe_out.2003.daily.005.pfb
reading flow for day 005 and converting from m^3/h to m^3/s
/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages/swe_out.2003.daily.006.pfb
reading flow for day 006 and converting from m^3/h to m^3/s
/hydrodata/temp/CONUS2_transfers/CONUS2/spinup_WY2003/averages/swe_out.2003.daily.007.pfb
reading fl

In [7]:
# Organize daily flow matched array with the same index and dates as the obs_data
pf_flow_match_df = pd.DataFrame(pf_flow_matched)
pf_flow_match_df = pf_flow_match_df.set_index(obs_data.index)
column_headers = list(obs_data.columns.values)
pf_flow_match_df.columns = column_headers
pf_flow_match_df.shape

(640, 365)

In [8]:
pf_flow_match_df_new = pf_flow_match_df.dropna()
pf_flow_match_df_new

,2002-10-01,2002-10-02,2002-10-03,2002-10-04,2002-10-05,2002-10-06,2002-10-07,2002-10-08,2002-10-09,2002-10-10,...,2003-09-21,2003-09-22,2003-09-23,2003-09-24,2003-09-25,2003-09-26,2003-09-27,2003-09-28,2003-09-29,2003-09-30
site_id,,,,,,,,,,,,,,,,,,,,,
1000:OR:SNTL,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005:CO:SNTL,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1006:NV:SNTL,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1008:MT:SNTL,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009:MT:SNTL,0.0,0.0,0.0,0.925932,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984:WA:SNTL,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
985:WA:SNTL,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
988:ID:SNTL,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
obs_data_new = obs_data[obs_data.index.isin(pf_flow_match_df_new.index)]
obs_data_new

,2002-10-01,2002-10-02,2002-10-03,2002-10-04,2002-10-05,2002-10-06,2002-10-07,2002-10-08,2002-10-09,2002-10-10,...,2003-09-21,2003-09-22,2003-09-23,2003-09-24,2003-09-25,2003-09-26,2003-09-27,2003-09-28,2003-09-29,2003-09-30
site_id,,,,,,,,,,,,,,,,,,,,,
1000:OR:SNTL,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005:CO:SNTL,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1006:NV:SNTL,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1008:MT:SNTL,0.0,0.0,0.0,0.0,2.54,7.62,2.54,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009:MT:SNTL,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984:WA:SNTL,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
985:WA:SNTL,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
988:ID:SNTL,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
metadata_new = metadata[metadata.index.isin(pf_flow_match_df_new.index)]
metadata_new

,site_type,agency,site_name,site_id_agency,site_query_url,date_metadata_last_updated,latitude,longitude,state,huc,...,file_path,county_name,elevation,shef_id,acton_id,conus1_x,conus1_y,conus2_x,conus2_y,num_obs
site_id,,,,,,,,,,,,,,,,,,,,,
1000:OR:SNTL,SNOTEL station,NRCS,Annie Springs,1000:OR:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,42.87007,-122.16518,OR,180102030101,...,/hydrodata/national_obs/swe/data/daily/1000:OR...,Klamath,6010.0,ANSO3,22G06S,NaN,NaN,258,2286,365
1005:CO:SNTL,SNOTEL station,NRCS,Ute Creek,1005:CO:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,37.61480,-105.37322,CO,130100020604,...,/hydrodata/national_obs/swe/data/daily/1005:CO...,Costilla,10734.0,UTCC2,05M17S,1063.0,495.0,1490,1448,365
1006:NV:SNTL,SNOTEL station,NRCS,Lewis Peak,1006:NV:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,40.35720,-116.86470,NV,160401070903,...,/hydrodata/national_obs/swe/data/daily/1006:NV...,Lander,7370.0,LWPN2,16J01S,137.0,957.0,592,1908,365
1008:MT:SNTL,SNOTEL station,NRCS,Onion Park,1008:MT:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,46.91348,-110.85360,MT,100301030801,...,/hydrodata/national_obs/swe/data/daily/1008:MT...,Meagher,7410.0,ONPM8,10C22S,753.0,1575.0,1196,2499,365
1009:MT:SNTL,SNOTEL station,NRCS,Stringer Creek,1009:MT:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,46.92690,-110.90198,MT,100301030801,...,/hydrodata/national_obs/swe/data/daily/1009:MT...,Meagher,6550.0,STCM8,10C23S,750.0,1577.0,1193,2501,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984:WA:SNTL,SNOTEL station,NRCS,Spruce Springs,984:WA:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,46.18287,-117.54155,WA,170601030201,...,/hydrodata/national_obs/swe/data/daily/984:WA:...,Garfield,5700.0,SPGW1,17C04S,233.0,1598.0,697,2528,365
985:WA:SNTL,SNOTEL station,NRCS,Sourdough Gulch,985:WA:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,46.23718,-117.39438,WA,170601030202,...,/hydrodata/national_obs/swe/data/daily/985:WA:...,Asotin,4000.0,SGUW1,17C06S,245.0,1601.0,709,2531,365
988:ID:SNTL,SNOTEL station,NRCS,Hidden Lake,988:ID:SNTL,https://wcc.sc.egov.usda.gov/awdbWebService/we...,2023-03-07,48.89370,-116.75748,ID,170101040502,...,/hydrodata/national_obs/swe/data/daily/988:ID:...,Boundary,5040.0,HDLI1,16A12S,361.0,1879.0,825,2796,365


In [12]:
# save matched daily average flow and metadata as csv's
pf_flow_match_df_new.to_csv(f'{organized_dir}/PFCONUS2_Daily_matched_SWE_mm_{water_year}.csv', sep = ",")
obs_data_new.to_csv(f'{organized_dir}/SNOTEL_Daily_matched_SWE_mm_{water_year}.csv', sep = ",")
metadata_new.to_csv(f'{organized_dir}/SNOTEL_metadata_matched_SWE_mm_{water_year}.csv', sep = ",")

In [ ]:
# # Calculate Annual Average Flow for PFCLM and save
# pf_flow_avg = np.mean(pf_flow_array, axis = 0) #monthly average 
# pf_flow_avg.shape
# np.savetxt(f'{organized_dir}/PFCONUS2_Annual_Avg_Flow_cms_WY2003.csv', pf_flow_avg, delimiter=",")

In [ ]:
pf_flow_array.shape

In [ ]:
pf_flow_match.shape